In [106]:
import pandas as pd
import numpy as np
def main():
    df_comp = pd.read_csv ("../company.csv",encoding='utf8',sep=";")
    df_comp = pd.DataFrame(df_comp,columns=["Siret","Raison sociale","Adresse normée ligne 4","Ville","Code postal",
                                      "Libellé activité","Code activité","Date de création","Nom dirigeant principal","Prénom dirigeant principal",
                                      "Tranche Effectif INSEE","Tranche Effectif établissement","Téléphone","Email","Chiffre d'affaires"])
    print(df_comp["Email"])
    
    df_cont = pd.read_csv ("../contact.csv",encoding='utf8',sep=";")
    df_cont = pd.DataFrame(df_cont,columns=[
        "Civilité","Nom","Prénom","Fonction","Raison sociale","Téléphone","Email"
    ])
    # print(df_cont[df_comp["Nom"].duplicated()==True])
        
    # print(df_cont)
    return df_comp,df_cont
df_comp,df_cont = main()

0        contact@victorhugohotel.com
1                                NaN
2                                NaN
3                                NaN
4           contact@pmc-decoupage.fr
                    ...             
29797                            NaN
29798    reception@hotelmagellan.com
29799                            NaN
29800                            NaN
29801                            NaN
Name: Email, Length: 29802, dtype: object


In [107]:
import re
import math

def is_nan(x):
    return math.isnan(x)

def format_phone(phone):
    if(not is_nan(phone)):
        phone = int(phone)
        phone = '0'+str(phone)
        # Enlève tous les caractères qui ne sont pas des chiffres
        prefixe = '+33'
        numero = phone.replace(' ', '')
        numero = numero.replace('-', '')
        numero = numero.replace('.', '')
        numero = prefixe + numero[1:]
        numero_formatte = numero[0:3] + ' ' + numero[3]+' '+' '.join([numero[i:i+2] for i in range(4, len(numero), 2)])
        return numero_formatte
    else:
        return phone

In [108]:
df_comp['Téléphone']

0        145531491.0
1                NaN
2        147952695.0
3        153338200.0
4        164255087.0
            ...     
29797            NaN
29798    145724451.0
29799    148265706.0
29800    146266030.0
29801    169015447.0
Name: Téléphone, Length: 29802, dtype: float64

In [109]:
df_cont['Téléphone'] = df_cont['Téléphone'].apply(format_phone)
df_comp['Téléphone'] = df_comp['Téléphone'].apply(format_phone)

In [110]:
df_cont['Téléphone']

0        +33 1 45 53 14 91
1        +33 1 45 53 14 91
2        +33 1 45 53 14 91
3        +33 1 45 53 14 91
4        +33 1 45 53 14 91
               ...        
32411    +33 1 45 72 44 51
32412    +33 1 45 72 44 51
32413    +33 1 48 26 57 06
32414    +33 1 46 26 60 30
32415    +33 1 69 01 54 47
Name: Téléphone, Length: 32416, dtype: object

In [111]:
def getAnneeCreation(date):
    if(len(date)> 0):
        date = str(date)
        date = date.split("/")
        return date[-1]
    else:
        return date
    


In [112]:
df_comp['Date de création'] = df_comp['Date de création'].apply(getAnneeCreation)
df_comp = df_comp.assign(responsable="NAN")
df_comp = df_comp.replace(np.nan,'')
df_comp = df_comp.replace('nan','')
df_comp = df_comp.replace('NaN','')
df_comp['responsable'] = df_comp['Nom dirigeant principal'].astype(str) +" "+ df_comp["Prénom dirigeant principal"]
df_comp = df_comp.drop(['Nom dirigeant principal','Prénom dirigeant principal'],axis=1)

In [113]:
df_comp

,Siret,Raison sociale,Adresse normée ligne 4,Ville,Code postal,Libellé activité,Code activité,Date de création,Tranche Effectif INSEE,Tranche Effectif établissement,Téléphone,Email,Chiffre d'affaires,responsable
0,4725056800020,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,19 RUE COPERNIC,PARIS 16,75116,Hôtels et hébergement similaire,5510Z,1974,10 à 19,10 à 19,+33 1 45 53 14 91,contact@victorhugohotel.com,1080941,Blanc Jacques
1,6080209700067,ETABLISSEMENTS TIANO,6 RUE DU GENERAL DELESTRAINT,PARIS 16,75016,"Intermédiaires du commerce en machines, équipe...",4614Z,1960,1 ou 2,,,,619500,TIANO ALFRED JACQUES
2,6080321000032,BENEX INTERNATIONAL,15 RUE DU DIX NEUF JANVIER,GARCHES,92380,Intermédiaires du commerce en bois et matériau...,4613Z,1984,1 ou 2,1 ou 2,+33 1 47 95 26 95,,766565,BARICHELLA GILBERT
3,6520036200062,JEAN-CHARLES BROSSEAU SAS,129 AV DAUMESNIL,PARIS 12,75012,Fabrication de parfums et de produits pour la ...,2042Z,1964,1 ou 2,1 ou 2,+33 1 53 33 82 00,,548675,BROSSEAU Benoit
4,30006868100042,PRECISION MECANIQUE DE COUBRON,39 RUE AMPERE,GRETZ-ARMAINVILLIERS,77220,Mécanique industrielle,2562B,1974,3 à 5,3 à 5,+33 1 64 25 50 87,contact@pmc-decoupage.fr,719449,HUMBERT Sébastien David
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29797,99811550500013,SOCIETE MATIGNON SAINT-HONORE,34 AV MATIGNON,PARIS 8,75008,Commerce de détail de biens d'occasion en magasin,4779Z,1976,1 ou 2,1 ou 2,,,588500,Malfet Fabrice
29798,99817510300014,SOCIETE HOTELIERE PARIS-OUEST,17 RUE JEAN BAPTISTE DUMAS,PARIS 17,75017,Hôtels et hébergement similaire,5510Z,1976,10 à 19,10 à 19,+33 1 45 72 44 51,reception@hotelmagellan.com,589484,PROGINOV CONSEIL (SARL)
29799,99841701800038,ENTREPRISE D ETANCHEITE NABAIS,19 RUE GAMBETTA,STAINS,93240,Travaux d'étanchéification,4399A,1976,10 à 19,10 à 19,+33 1 48 26 57 06,,1127578,DE BASTOS Antonio
29800,99862092600043,SARL DOMEAU CONCEPT,90 GRANDE RUE,SEVRES,92310,Fabrication de serrures et de ferrures,2572Z,1900,3 à 5,3 à 5,+33 1 46 26 60 30,,868987,WINZELBERG Olivier


In [114]:
df_cont

,Civilité,Nom,Prénom,Fonction,Raison sociale,Téléphone,Email
0,M.,QUENNESSON,Olivier,Dir. des Ressources Humaines,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,+33 1 45 53 14 91,contact@victorhugohotel.com
1,M.,QUENNESSON,Olivier,Dir Administratif et Financier,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,+33 1 45 53 14 91,contact@victorhugohotel.com
2,MLE,ARTIERES,Maguelone,Directeur Marketing,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,+33 1 45 53 14 91,contact@victorhugohotel.com
3,MLE,ARTIERES,Maguelone,Directeur Commercial,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,+33 1 45 53 14 91,contact@victorhugohotel.com
4,M.,Blanc,Jacques,Président,SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOT...,+33 1 45 53 14 91,contact@victorhugohotel.com
...,...,...,...,...,...,...,...
32411,MME,DEQUAIRE,Hélène,Dir. des Ressources Humaines,SOCIETE HOTELIERE PARIS-OUEST,+33 1 45 72 44 51,reception@hotelmagellan.com
32412,M.,PROGINOV CONSEIL (SARL),NaN,Président,SOCIETE HOTELIERE PARIS-OUEST,+33 1 45 72 44 51,reception@hotelmagellan.com
32413,M.,DE BASTOS,Antonio,Gérant,ENTREPRISE D ETANCHEITE NABAIS,+33 1 48 26 57 06,NaN
32414,M.,WINZELBERG,Olivier,Gérant,SARL DOMEAU CONCEPT,+33 1 46 26 60 30,NaN


In [6]:
for index,z in df_cont[df_cont['Raison sociale']=="SOCIETE HOTELIERE PARIS-OUEST"].iterrows():
    print(z)

Civilité                                     M.
Nom                                       STOLL
Prénom                                    Alain
Fonction                    Directeur Financier
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 1 45 72 44 51
Email               reception@hotelmagellan.com
Name: 32409, dtype: object
Civilité                                    MME
Nom                                    DEQUAIRE
Prénom                                   Hélène
Fonction                   Directeur des Achats
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 1 45 72 44 51
Email               reception@hotelmagellan.com
Name: 32410, dtype: object
Civilité                                    MME
Nom                                    DEQUAIRE
Prénom                                   Hélène
Fonction           Dir. des Ressources Humaines
Raison sociale    SOCIETE HOTELIERE PARIS-OUEST
Téléphone                     +33 

In [186]:
#TEST API
import requests
import json
import pprint


# Définissez l'URL de l'API d'importation
df_comp = df_comp.fillna("lol")
df_cont = df_cont.fillna("lol")
entreprises=[]
for index,x in df_comp.head(10).iterrows():
      my_company = {
        "type": "company",
        "properties": {
          "name": {"value": x['Raison sociale']},
          "Type principal":{"value" : x['Libellé activité']},
          "Téléphone":{"value":str(x['Téléphone'])},
          "Email":{"value":x['Email']}
        }
      }
      entreprises.append(my_company)
      for index,z in df_cont.head(10)[df_cont['Raison sociale']==x['Raison sociale']].iterrows():
            my_contact = {
              "type": "contact",
              "properties": {
                "Prenom":{"value":z['Prénom']},
                "Nom":{"value":z['Nom']},
                "Téléphone":{"value":str(z['Téléphone'])},
                "company":{"value":z['Raison sociale']}
                }
            }
            entreprises.append(my_contact) 

# Créez le corps de la requête
data = {
  "importId": "my_import",
  "createIfNotExists": "true",
  "inputs": entreprises
}
# pprint.pprint(data)
data = json.dumps(data,ensure_ascii=False)
pprint.pprint(data)
print(data)

('{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": '
 '"company", "properties": {"name": {"value": "SOCIETE DE PARTICIPATION ET '
 'D\'EXPLOITATION HOTELIERE SOPAROTEL"}, "Type principal": {"value": "Hôtels '
 'et hébergement similaire"}, "Téléphone": {"value": "+33 1 45 53 14 91"}, '
 '"Email": {"value": "contact@victorhugohotel.com"}}}, {"type": "contact", '
 '"properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": '
 '"QUENNESSON"}, "Téléphone": {"value": "+33 1 45 53 14 91"}, "company": '
 '{"value": "SOCIETE DE PARTICIPATION ET D\'EXPLOITATION HOTELIERE '
 'SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": '
 '"Olivier"}, "Nom": {"value": "QUENNESSON"}, "Téléphone": {"value": "+33 1 45 '
 '53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET '
 'D\'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": '
 '{"Prenom": {"value": "Maguelone"}, "Nom": {"value": "ARTIERES"}, '
 '"Téléphone": {"value": "+33 

c:\Users\Icet0\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [202]:
data = {
  "importId": "my_import",
  "createIfNotExists": "true",
  "inputs": [
    {
      "type": "company",
      "properties": {
        "name": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" },
        "Type principal": { "value": "Hôtels et hébergement similaire" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "Email": { "value": "contact@victorhugohotel.com" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Olivier" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Olivier" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
        {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Maguelone" },
        "Nom": { "value": "ARTIERES" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Maguelone" },
        "Nom": { "value": "ARTIERES" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Jacques" },
        "Nom": { "value": "Blanc" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "Marie-Hélène" },
        "Nom": { "value": "QUENNESSON" },
        "Téléphone": { "value": "+33 1 45 53 14 91" },
        "company": { "value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL" }
      }
    },
    {
      "type": "company",
      "properties": {
        "name": { "value": "ETABLISSEMENTS TIANO" },
        "Type principal": { "value": "Intermédiaires du commerce en machines, équipements industriels, navires et avions" },
        "Téléphone": { "value": "de" },
        "Email": { "value": "de" }
      }
    },
    {
      "type": "contact",
      "properties": {
        "Prenom": { "value": "ALFRED JACQUES" },
        "Nom": { "value": "TIANO" },
        "Téléphone": { "value": "de" },
        "company": { "value": "ETABLISSEMENTS TIANO" }
      }
    },
  ]}
# transforme le dictionnaire en json
data_string = json.dumps(data)
payload = {"importRequest": data}
print(payload["importRequest"])
# Envoyez la requête POST à l'API d'importation
# Définissez votre clé API d'HubSpot
api_key = "pat-eu1-91242dbb-e9d6-4b77-852f-fdd31151e049"
url = "https://api.hubapi.com/crm/v3/imports"
headers = {
  # 'content-type': 'multipart/form-data',
  # 'accept': 'application/json',
  'Authorization': 'Bearer %s' % api_key
}


response = requests.request("POST", url, data=payload,headers=headers)
print(response.encoding)
print(response.text.encode('utf8'))
print(response.status_code)
# Affichez le statut de la réponse
print(response.status_code)

# Si la réponse est un succès (code 200), affichez le message de réussite
if response.status_code == 200:
    print('Import réussi !')
# Sinon, affichez le message d'erreur
else:  
    print('Erreur lors de l\'import !')
    print(response.json())


{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": "company", "properties": {"name": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}, "Type principal": {"value": "H\u00f4tels et h\u00e9bergement similaire"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "Email": {"value": "contact@victorhugohotel.com"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": "QUENNESSON"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Olivier"}, "Nom": {"value": "QUENNESSON"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53 14 91"}, "company": {"value": "SOCIETE DE PARTICIPATION ET D'EXPLOITATION HOTELIERE SOPAROTEL"}}}, {"type": "contact", "properties": {"Prenom": {"value": "Maguelone"}, "Nom": {"value": "ARTIERES"}, "T\u00e9l\u00e9phone": {"value": "+33 1 45 53

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [205]:
import requests

# Définissez votre jeton d'API et l'URL de l'API d'import
api_key = "pat-eu1-91242dbb-e9d6-4b77-852f-fdd31151e049"
import_url = "https://api.hubapi.com/crm/v3/imports"

# Préparez le corps de la requête au format JSON
import_data = {
    "importId": "my_import",
    "createIfNotExists": "true",
    "inputs": [
        {
            "type": "company",
            "properties": {
                "name": { "value": "Acme Corp" },
                "phone": { "value": "+1 555 555 5555" }
            }
        },
        {
            "type": "contact",
            "properties": {
                "firstName": { "value": "John" },
                "lastName": { "value": "Doe" },
                "phone": { "value": "+1 555 555 5555" },
                "company": { "value": "Acme Corp" }
            }
        }
    ]
}

data = json.dumps(import_data)
# Envoyez la requête HTTP POST à l'API d'import
headers = { "Authorization": f"Bearer {api_key}" }
response = requests.post(import_url, json=data, headers=headers)

# Affichez le statut de la réponse et le corps de la réponse (s'il y en a un)
print(response.status_code)
print(response.text)


{"importId": "my_import", "createIfNotExists": "true", "inputs": [{"type": "company", "properties": {"name": {"value": "Acme Corp"}, "phone": {"value": "+1 555 555 5555"}}}, {"type": "contact", "properties": {"firstName": {"value": "John"}, "lastName": {"value": "Doe"}, "phone": {"value": "+1 555 555 5555"}, "company": {"value": "Acme Corp"}}}]}
415
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
<title>Error 415 Unsupported Media Type</title>
</head>
<body><h2>HTTP ERROR 415</h2>
<p>Reason:
<pre>    Unsupported Media Type</pre></p>
</body>
</html>



## TRY POUR BENOIT 


In [64]:
import pandas as pd
import numpy as np
import math


In [65]:
#ouvrir le fichier excel nocrm-leads(benoit).csv
df = pd.read_csv('../nocrm-leads(benoit).csv')
df

d:\Travail\Polytech 4A\STAGE\MediaStart\listing\env\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (27,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Lead,ID,SOCIETE,Prénom,Nom,Téléphone,E-mail,Adresse,CP,VILLE,...,Reminder_note,Created_at,First_activity,Closed_at,Estimated_closing_date,Last_update,Client_folder,tags,Created_from,Description
0,CH OSCAR VALETTE,21750799,NaN,NaN,NaN,0688963503,oscarvalette@gmail.com,NaN,NaN,NaN,...,NaN,2022-10-19 11:32,2022-10-19 11:43,NaN,NaN,2023-01-13 09:56,NaN,"Victor,site",sales@digi-certif.com,Valette\nOscar\nCH\nComment nous avez vous con...
1,Institut dermo'k,17742327,Institut dermo'k,NaN,Kassouri,0635561095,NaN,NaN,NaN,NaN,...,NaN,2021-11-09 15:12,2021-11-15 11:40,NaN,NaN,2022-09-13 14:43,NaN,"Hugo,site",NaN,Email : moha661@hotmail.fr\n
2,Madame MATHILDE CHAUVOT,19608235,NaN,NaN,NaN,+33 6 61 50 19 63,NaN,2080 ROUTE DES CRUS DU BEAUJOLAIS,71570.0,ST AMOUR BELLEVUE,...,NaN,2022-05-03 14:52,2022-05-03 14:51,NaN,NaN,2023-02-24 09:36,NaN,Victor,spreadsheet,8.51191585E8 : 8.82584303E8\n
3,Mhammed EL HADADI,22480367,NaN,NaN,NaN,0782544237,elhadadi_mhammed@yahoo.fr,NaN,NaN,NaN,...,NaN,2022-12-19 13:27,2022-12-19 13:52,NaN,NaN,2022-12-19 13:58,NaN,"Victor,site",sales@digi-certif.com,EL HADADI\nMhammed\nMhammed EL HADADI\nComment...
4,FORMA'NGO,22760325,NaN,NaN,NaN,+33 6 48 05 26 50,NaN,NaN,NaN,NaN,...,NaN,2023-01-18 11:07,2023-01-18 11:06,NaN,NaN,2023-01-18 11:08,NaN,Matthieu,spreadsheet,Numéro Déclaration Activité : 11941150994\nAdr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26774,ACTIONS FORMATIONS,13245752,NaN,NaN,NaN,565590764,contact@actionsformations.fr,NaN,NaN,NaN,...,NaN,2020-10-20 14:28,2020-10-20 14:28,2020-10-20,NaN,2020-10-20 14:35,NaN,NaN,spreadsheet,WEBSITE : http://www.actionsformations.fr/\n
26775,ACTION REFLEXO FORMATION,13245681,NaN,Eric,Gimbert,683974067,NaN,NaN,NaN,NaN,...,NaN,2020-10-20 14:23,2020-10-20 14:23,2020-10-20,NaN,2020-10-20 14:34,NaN,NaN,spreadsheet,WEBSITE : https://action-reflexo.fr/\n
26776,ACTIONS CONSEILS,13245555,NaN,NaN,NaN,660778889,NaN,NaN,NaN,NaN,...,NaN,2020-10-20 14:13,2020-10-20 14:13,2020-10-20,NaN,2020-10-20 14:35,NaN,NaN,spreadsheet,WEBSITE : https://www.actions-conseils.fr/cont...
26777,ACTION RESILIENCE,13244381,NaN,NaN,NaN,970407077,NaN,NaN,NaN,NaN,...,NaN,2020-10-20 12:16,2020-10-20 12:16,2021-01-13,NaN,2021-01-13 10:44,NaN,NaN,spreadsheet,WEBSITE : \n


In [66]:
df['Description'] = df['Description'].replace(r'\\n','\n', regex=True)
df['Description'] = df['Description'].replace(r'\xa0',' ', regex=True)


In [67]:
import re

def extract_email(text):
    """Fonction pour extraire une adresse e-mail d'une chaîne de caractères"""
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    # rechercher le premier match dans le texte
    if (text == np.nan or type(text) == float):
        text = ""
    match = re.search(email_regex, text)
    if match:

        return match.group(0)
    else:
        return None
    
    
def format_phone(phone):
    if(not (type(phone)==float or phone == np.nan or phone == None or phone == "nan" or phone == "None" or phone == "")):
        if(phone[0] == "+"):
            return phone
        phone = int(phone)
        phone = '0'+str(phone)
        print(phone)
        # Enlève tous les caractères qui ne sont pas des chiffres
        prefixe = '+33'
        numero = phone.replace(' ', '')
        numero = numero.replace('-', '')
        numero = numero.replace('.', '')
        numero = prefixe + numero[1:]
        print("numero",numero)
        numero_formatte = numero[0:3] + ' ' + numero[3]+' '+' '.join([numero[i:i+2] for i in range(4, len(numero), 2)])
        print("numero_formatte",numero_formatte)
        return numero_formatte
    else:
        return phone

In [68]:
df['Description'][245:255]
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
match = re.search(email_regex, df['Description'][245])
match.group(0)

'sayici.ebru@gmail.com'

In [69]:
dfSave = df.copy()


In [71]:
df['NewE-mail'] = df['Description'].apply(extract_email)
print(df['NewE-mail'][245])
df['E-mail'] = df['E-mail'].fillna('')
df['E-mail'] = (df['E-mail'] + '\n' + df['NewE-mail'].fillna('')).reset_index(drop=True)
print(df['E-mail'][240:255])

# df = df.drop('NewE-mail', axis=1)

# Afficher les premières lignes du dataframe pour vérifier les résultats
print(df['E-mail'].head(10))
print(dfSave['E-mail'].head(10))

sayici.ebru@gmail.com
240                                 NaN
241                                 NaN
242                      glj@sfr.fr\n\n
243                contact@idyt.com\n\n
244    valerie.gourliau@outlook.com\n\n
245                                 NaN
246         nath.mdfrance@gmail.com\n\n
247       ludovic.brouty@hotmail.fr\n\n
248                                 NaN
249         hanane34200@outlook.com\n\n
250                                 NaN
251        lesautermaelle@gmail.com\n\n
252                                 NaN
253                                 NaN
254                                 NaN
Name: E-mail, dtype: object
0                           oscarvalette@gmail.com\n\n
1                                                  NaN
2                                                  NaN
3                        elhadadi_mhammed@yahoo.fr\n\n
4                                                  NaN
5    infos.nugreen@gmail.com\nvanessa@nugreenhomech...
6                   

In [74]:
df['NewE-mail'].fillna('')
var = 'monPremiermail@mail.com'
var = var + '\n' + df['NewE-mail'][245]
df['E-mail'][245] + '\n' + df['NewE-mail'][245]
df['E-mail'][245]

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [159]:
df.to_csv("nouveau_fichier.csv", index=False)


In [150]:
df['Téléphone'] = df['Téléphone'].replace(r'\xa0','', regex=True)
df['Téléphone'][0]

'0688963503'

In [151]:
df["Téléphone"] = df["Téléphone"].fillna('')
df['Téléphone'] = df['Téléphone'].apply(format_phone)
df

0688963503
numero +33688963503
numero_formatte +33 6 88 96 35 03
0635561095
numero +33635561095
numero_formatte +33 6 35 56 10 95
0782544237
numero +33782544237
numero_formatte +33 7 82 54 42 37
0606968666
numero +33606968666
numero_formatte +33 6 06 96 86 66
0781211929
numero +33781211929
numero_formatte +33 7 81 21 19 29
0662269489
numero +33662269489
numero_formatte +33 6 62 26 94 89
0632204954
numero +33632204954
numero_formatte +33 6 32 20 49 54
0769502062
numero +33769502062
numero_formatte +33 7 69 50 20 62
0562616211
numero +33562616211
numero_formatte +33 5 62 61 62 11
0689230384
numero +33689230384
numero_formatte +33 6 89 23 03 84
0618365509
numero +33618365509
numero_formatte +33 6 18 36 55 09
0695358757
numero +33695358757
numero_formatte +33 6 95 35 87 57
0619764233
numero +33619764233
numero_formatte +33 6 19 76 42 33
0142471852
numero +33142471852
numero_formatte +33 1 42 47 18 52
0228244589
numero +33228244589
numero_formatte +33 2 28 24 45 89
0669082535
numero +336690

ValueError: invalid literal for int() with base 10: '04 72 68 60 90 / 04 72 68 99 89'

In [139]:
for i, row in df.iterrows():
    try:
        int(row['Téléphone'])
    except ValueError:
        print(f"Erreur de format dans la ligne {i}: {row['Téléphone']}")

Erreur de format dans la ligne 2: +33 6 61 50 19 63
Erreur de format dans la ligne 4: +33 6 48 05 26 50
Erreur de format dans la ligne 11: +33 6 19 57 48 52
Erreur de format dans la ligne 13: nan
Erreur de format dans la ligne 17: +33 6 47 82 38 79
Erreur de format dans la ligne 21: +33 9 81 63 38 92
Erreur de format dans la ligne 23: +33 7 81 75 73 09
Erreur de format dans la ligne 27: +33 6 22 76 94 47
Erreur de format dans la ligne 32: +33 6 83 19 84 92
Erreur de format dans la ligne 35: +33 6 11 52 68 03
Erreur de format dans la ligne 37: +33 7 56 80 31 56
Erreur de format dans la ligne 39: +33 6 20 75 35 28
Erreur de format dans la ligne 43: +33 6 16 96 54 43
Erreur de format dans la ligne 44: +33 6 73 28 96 87
Erreur de format dans la ligne 47: +33 9 74 18 93 90
Erreur de format dans la ligne 49: +33 6 83 22 55 68
Erreur de format dans la ligne 50: +33 6 72 64 93 38
Erreur de format dans la ligne 55: +33 7 63 65 44 55
Erreur de format dans la ligne 57: +33 6 87 73 41 67
Erreur de